<a href="https://colab.research.google.com/github/tinbrick007/YouTube-Summarizer/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging face Model training and summarization

In [ ]:
!pip install youtube-transcript-api
#!pip install groq
!pip install openai


from youtube_transcript_api import YouTubeTranscriptApi
#from openai import OpenAI
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline
import os

import csv
from urllib.parse import urlparse, parse_qs

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


nltk.download('vader_lexicon')

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import pandas as pd

creds, _ = default()
print(creds.get_cred_info)
#gc = gspread.authorize(creds)

# Load BART model and tokenizer once
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

count = 0

def convert_dict_to_document(data_dict, filename="output_document.txt"):
  """
  Converts a Python dictionary into a simple text document.

  Args:
    data_dict: The dictionary to convert.
    filename: The name of the output file.
  """
  with open(filename, 'w', encoding='utf-8') as f:
    for key, value in data_dict.items():
      f.write(f"--- {key} ---\n")
      f.write(f"{value}\n\n")
  print(f"Document created: {filename}")



def extract_video_id(youtube_url):
    if "youtu.be" in youtube_url:
        video_id = youtube_url.split("/")[-1].split("?")[0]
    elif "youtube.com" in youtube_url:
        video_id = youtube_url.split("v=")[-1].split("&")[0]
    else:
        raise ValueError("Invalid YouTube URL format")
    return video_id


def get_youtube_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        transcript = ' '.join([entry['text'] for entry in transcript_list])
        return transcript
    except Exception as e:
        count+=1
        raise Exception(f"Error fetching transcript: {e}")


def get_summary(transcript):
    inputs = tokenizer.encode("Summarize the following real estate transcript for potential buyers. "
    "Focus on key highlights, location advantages, and investment value:\n\n" + transcript,
                              return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs, max_length=300, min_length=100, length_penalty=1.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


SHEET_NAME = 'youtube_links.xlsx'
LINK_COLUMN = 'A'
TITLE_COLUMN = 'A'

myinsights = {} # Initialize myinsights here

try:
  df = pd.read_excel(SHEET_NAME, engine='openpyxl')
  # View the column names (optional)
  print("Available columns:", df.columns)
  links = df['Video URL'].dropna().tolist()

  #sheet = gc.open(SHEET_NAME).get_worksheet(0)
  # Get all values from the first row
  #print(f"\nProcessing link: {sheet.row_values(2)}")
  # Get all values from the link column
  #youtube_links = [sheet.col_values()]
  youtube_links = links
  #titles = [cell.value for cell in sheet.col_values(sheet.find(TITLE_COLUMN).col)]
  #youtube_links = ["https://www.youtube.com/watch?v=2KvL9lTcewA"]
  print(len(youtube_links))

  for link in youtube_links:

    if link: # Check if the cell is not empty
      print(f"\nProcessing link: {link}")
      # Extract video ID from the link
      try:
          url_data = urlparse(link)
          if url_data.hostname == 'youtu.be':
              video_id = extract_video_id(link)
          elif url_data.hostname in ('www.youtube.com', 'youtube.com'):
              video_id = extract_video_id(link)
          else:
              video_id = None
              print("Invalid YouTube link format.")

          if video_id:
            transcript = get_youtube_transcript(video_id)
            if transcript:
              print("Transcript fetched successfully. Getting insights from Groq...")
              insights = get_summary(transcript)
              #print("Insights :",insights)
              #print(insights)
              myinsights[video_id] = insights
              #convert_dict_to_document(myinsights, "youtube_insights.txt")
            else:
              print("Skipping Groq analysis due to missing transcript.")
          else:
              print("Could not extract video ID from the link.")

      except Exception as e:
          print(f"Error processing link {link}: {e}")

except Exception as e:
  print(f"Error accessing Google Sheet: {e}")

print(myinsights)
print(f"\Links unprocessed {count}")
#convert_dict_to_document(myinsights, "youtube_insights.txt")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.7 MB/s eta 0:00:00


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


# **Final Code-working**

In [ ]:
# prompt: code to take youtube video links  from a csv file in runtime and download caption through API and pass it on to grok LLM through its API to get some insights out of it
!pip install youtube-transcript-api
!pip install groq
!pip install openai


from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
import os

import csv
from urllib.parse import urlparse, parse_qs




def convert_dict_to_document(data_dict, filename="output_document.txt"):
  """
  Converts a Python dictionary into a simple text document.

  Args:
    data_dict: The dictionary to convert.
    filename: The name of the output file.
  """
  with open(filename, 'w', encoding='utf-8') as f:
    for key, value in data_dict.items():
      f.write(f"--- {key} ---\n")
      f.write(f"{value}\n\n")
  print(f"Document created: {filename}")



# Function to extract video ID from a YouTube link
def extract_video_id(youtube_url):
    """Extracts the video ID from a YouTube URL."""
    try:
        url_data = urlparse(youtube_url)
        if url_data.hostname == 'youtu.be':
            return url_data.path[1:]
        elif url_data.hostname in ('www.youtube.com', 'youtube.com'):
            query = parse_qs(url_data.query)
            return query.get('v', [None])[0]
        else:
            return None
    except Exception as e:
        print(f"Error extracting video ID from {youtube_url}: {e}")
        return None

# Replace 'your_youtube_links.csv' with the actual path to your CSV file
csv_file_path = 'youtube.csv'
myinsights = {}
# Assuming the CSV has one column with YouTube links
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        # Skip header row if present
        # next(csv_reader)

        for row in csv_reader:
            if row: # Check if the row is not empty
                youtube_link = row[0].strip() # Assuming the link is in the first column
                if youtube_link:
                    print(f"\nProcessing link: {youtube_link}")
                    video_id = extract_video_id(youtube_link)

                    if video_id:
                        transcript = get_youtube_transcript(video_id)
                        if transcript:
                            print("Transcript fetched successfully. Getting insights from Groq...")
                            insights = get_grok_insights(transcript)
                            myinsights[video_id]=insights
                            print("Insights from Groq:")
                            #print(insights)
                        else:
                            print("Skipping Groq analysis due to missing transcript.")
                    else:
                        print("Could not extract video ID from the link.")
            else:
                print("Skipping empty row.")

except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
except Exception as e:
    print(f"Error reading CSV file: {e}")



# Replace with your Groq API key
# You can get one from https://console.groq.com/keys
API_KEY = 'xai-vikB5twp4LqLyqOoB25NzF8Qn8yWIZg5OlyO4xIgut7uK66cVZQuvacmSpUdSvS55lRvApmFPLVyv0VB'
# Replace with the Groq model you want to use, e.g., "llama3-8b-8192" or "llama3-70b-8192"
GROQ_MODEL = 'grok-3-mini-beta'

client = OpenAI(api_key=API_KEY, base_url="https://api.x.ai/v1")

def get_youtube_transcript(video_id):
  """Fetches the transcript for a given YouTube video ID."""
  try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    transcript = ' '.join([entry['text'] for entry in transcript_list])
    return transcript
  except Exception as e:
    print(f"Could not fetch transcript for video ID {video_id}: {e}")
    return None

def get_grok_insights(transcript):
  """Sends the transcript to the Groq API for insights."""
  if transcript is None:
    return "No transcript available to analyze."

  try:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """
                You are a smart YouTube transcript summarizer. Your job is to read the full transcript of a video and return:

        1. A short summary of the overall video (5-6 sentences).
        2. Bullet-pointed highlights of key takeaways, structured and concise.
        3. (Optional) Timestamps if they are included in the transcript.

        Guidelines:
        - Focus on clarity and information value such as numbers and statistics mentioned.
        - Ignore filler, repetition, or casual phrases.
        - Highlight important ideas, tips, steps, or arguments.
        - If the transcript is long, break it into parts and summarize each before merging.

        If the transcript is very short, just extract the key ideas directly.

        NEVER summarize the YouTube title or guess content — only work from the transcript provided.
            """
            },
            {
                "role": "user",
                "content": "Give me the summary of this transcript given: " + " "+transcript,
            }
        ],
        model=GROQ_MODEL,
        temperature=0.1 # Adjust max tokens as needed
    )
    return chat_completion.choices[0].message.content
  except Exception as e:
    print(f"Error calling Groq API: {e}")
    return "Could not get insights from Groq."

convert_dict_to_document(myinsights, "youtube_insights.txt")






Processing link: URL
Could not extract video ID from the link.

Processing link: https://youtu.be/Kd26vLt8M2E
Could not fetch transcript for video ID Kd26vLt8M2E: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=Kd26vLt8M2E! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (Kd26vLt8M2E) transcripts are available in the following languages:

(MANUALLY CREATED)
None

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - ab ("Abkhazian")
 - aa ("Afar")
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - ba ("Bashkir")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - br ("Breton")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - 